In [1]:
import sys
import pickle
sys.path.append("tools/")

import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
#warnings.simplefilter("ignore", category=UndefinedMetricWarning)

from feature_format import featureFormat, targetFeatureSplit
from tester import test_classifier, dump_classifier_and_data

import pandas as pd
pd.options.mode.use_inf_as_na = True
import numpy as np
import re
import operator
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score, \
                                    cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

## Introduction

By 2002, Enron, one of America's most successful companies of the 1990s and 2000s, had been suspended from the New York Stock Exchange and was being investigated by the Department of Justice.  Once trading at a peak of \$90.75, Enron's shares plummeted to $0.26 on December 2, 2001, when it declared bankruptcy.  At the time, Enron’s bankruptcy was the largest in U.S. history.  As a result, many executives at Enron were indicted and sentenced to prison time.  Enron used high-risk accounting practices that were meant to make the organization look more successful that it actually was in order to defraud shareholders.   Enron used mark-to-market accounting which used projected future earnings to measure the value of an asset, and if its value did not meet predictions they would hide the financial losses by assigning them to a shell company.

The goal of this project is to determine from the financial and email metadata features in the data set which of the employees are persons of interest in the fraud case against Enron.  Persons of interest are defiend as indiviuals who were indicted in the fraud, reached a settlement or plea deal with the government, or testified in exchange for immunity.

## Loading the Data

In [2]:
# Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "rb") as data_file:
    data_dict = pickle.load(data_file)

## Data Exploration

In [3]:
# Load the data into a dataframe for exploration
df = pd.DataFrame.from_dict(data_dict).T
df.head()

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
ALLEN PHILLIP K,4175000,2869717,-3081055,NaN,phillip.allen@enron.com,1729541,13868,2195,47,65,...,304805,152,False,126027,-126027,201955,1407,2902,4484442,1729541
BADUM JAMES P,NaN,178980,NaN,NaN,NaN,257817,3486,NaN,NaN,NaN,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,182466,257817
BANNANTINE JAMES M,NaN,NaN,-5104,NaN,james.bannantine@enron.com,4046157,56301,29,39,0,...,NaN,864523,False,1757552,-560222,477,465,566,916197,5243487
BAXTER JOHN C,1200000,1295738,-1386055,NaN,NaN,6680544,11200,NaN,NaN,NaN,...,1586055,2660303,False,3942714,NaN,267102,NaN,NaN,5634343,10623258
BAY FRANKLIN R,400000,260455,-201641,NaN,frank.bay@enron.com,NaN,129142,NaN,NaN,NaN,...,NaN,69,False,145796,-82782,239671,NaN,NaN,827696,63014


In [4]:
# Number of records
len(df)

146

In [5]:
# Replace 'NaN' in email_address field with empty string
df['email_address'].replace(to_replace='NaN', value = '', inplace=True)

# Replace all other 'NaN' with NaN 
df.replace(to_replace='NaN', value = np.nan, inplace=True)
df.head()

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
ALLEN PHILLIP K,4175000.0,2869717.0,-3081055.0,NaN,phillip.allen@enron.com,1729541.0,13868.0,2195.0,47.0,65.0,...,304805.0,152.0,False,126027.0,-126027.0,201955.0,1407.0,2902.0,4484442.0,1729541.0
BADUM JAMES P,NaN,178980.0,NaN,NaN,,257817.0,3486.0,NaN,NaN,NaN,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,182466.0,257817.0
BANNANTINE JAMES M,NaN,NaN,-5104.0,NaN,james.bannantine@enron.com,4046157.0,56301.0,29.0,39.0,0.0,...,NaN,864523.0,False,1757552.0,-560222.0,477.0,465.0,566.0,916197.0,5243487.0
BAXTER JOHN C,1200000.0,1295738.0,-1386055.0,NaN,,6680544.0,11200.0,NaN,NaN,NaN,...,1586055.0,2660303.0,False,3942714.0,NaN,267102.0,NaN,NaN,5634343.0,10623258.0
BAY FRANKLIN R,400000.0,260455.0,-201641.0,NaN,frank.bay@enron.com,NaN,129142.0,NaN,NaN,NaN,...,NaN,69.0,False,145796.0,-82782.0,239671.0,NaN,NaN,827696.0,63014.0


In [6]:
df = df.apply(lambda x: pd.to_numeric(x, errors='ignore'))
df.dtypes

bonus                        float64
deferral_payments            float64
deferred_income              float64
director_fees                float64
email_address                 object
exercised_stock_options      float64
expenses                     float64
from_messages                float64
from_poi_to_this_person      float64
from_this_person_to_poi      float64
loan_advances                float64
long_term_incentive          float64
other                        float64
poi                             bool
restricted_stock             float64
restricted_stock_deferred    float64
salary                       float64
shared_receipt_with_poi      float64
to_messages                  float64
total_payments               float64
total_stock_value            float64
dtype: object

In [7]:
# Number of features
print(len(list(df.columns)))

21


There are a total of 21 fields, 14 financial features, 6 features concerning email metadata, and the target field poi.

**financial features**
* bonus
* deferral_payments
* deferred_income
* director_fees
* exercised_stock_options
* expenses
* loan_advances
* long_term_incentive
* other
* restricted_stock
* restricted_stock_deferred
* salary
* total_payments
* total_stock_value

**email features**
* email_address
* from_messages
* from_poi_to_this_person
* from_this_person_to_poi
* shared_receipt_with_poi
* to_messages

In [8]:
target = 'poi'

email_features = ['email_address',
                  'from_messages',
                  'from_poi_to_this_person',
                  'from_this_person_to_poi',
                  'shared_receipt_with_poi',
                  'to_messages']

financial_features = ['bonus',
                      'deferral_payments',
                      'deferred_income',
                      'director_fees',
                      'exercised_stock_options',
                      'expenses',
                      'loan_advances',
                      'long_term_incentive',
                      'other',
                      'restricted_stock',
                      'restricted_stock_deferred',
                      'salary',
                      'total_payments',
                      'total_stock_value']

### Removing records

In [9]:
# Appropriate formats for names
name_fmt_1 = '\w+ \w+ \w'
name_fmt_2 = '\w+ \w+'

# Print names that do not follow either of these formats
[p for p in df.index if not (re.fullmatch(name_fmt_1, p) or re.fullmatch(name_fmt_2, p))]

['BLAKE JR. NORMAN P',
 'BOWEN JR RAYMOND M',
 'DERRICK JR. JAMES V',
 'DONAHUE JR JEFFREY M',
 'GARLAND C KEVIN',
 'GLISAN JR BEN F',
 'OVERDYKE JR JERE C',
 'PEREIRA PAULO V. FERRAZ',
 'SULLIVAN-SHAKLOVITZ COLLEEN',
 'THE TRAVEL AGENCY IN THE PARK',
 'TOTAL',
 'WALLS JR ROBERT H',
 'WHITE JR THOMAS E',
 'WINOKUR JR. HERBERT S',
 'YEAGER F SCOTT']

In [10]:
# Remove periods from names to make them consistent
df.index = df.index.map(lambda x: x.replace('.', ''))

[p for p in df.index if not (re.fullmatch(name_fmt_1, p) or re.fullmatch(name_fmt_2, p))]

['BLAKE JR NORMAN P',
 'BOWEN JR RAYMOND M',
 'DERRICK JR JAMES V',
 'DONAHUE JR JEFFREY M',
 'GARLAND C KEVIN',
 'GLISAN JR BEN F',
 'OVERDYKE JR JERE C',
 'PEREIRA PAULO V FERRAZ',
 'SULLIVAN-SHAKLOVITZ COLLEEN',
 'THE TRAVEL AGENCY IN THE PARK',
 'TOTAL',
 'WALLS JR ROBERT H',
 'WHITE JR THOMAS E',
 'WINOKUR JR HERBERT S',
 'YEAGER F SCOTT']

Apart from the names with 'JR', additional middle names, a first initial, or a hyphenated last name, which are also acceptable, there are the records 'TOTAL' and 'THE TRAVEL AGENCY IN THE PARK'.  Obviously these records were included in the data set in error since they do not represent employees.  Below, inspection of the data confirms that 'TOTAL' is an aggregate row of all employees and was erroneously included in the data.  These records will be removed from the data.

In [11]:
df['salary'].idxmax()

'TOTAL'

In [12]:
df['salary'].max() == df['salary'].sum() - df['salary'].max()

True

In [13]:
# Drop non-employee records
df.drop(['THE TRAVEL AGENCY IN THE PARK', 'TOTAL'], inplace = True)

I also chose to remove 'email_address' from the features since it cannot and will not be used to train the model.

In [14]:
# Remove email since it is basically equivalent to an ID and cannot be used in a machine learning model
df.drop('email_address', axis = 1, inplace = True)

### Missing Values

In [15]:
df.isna().sum()

bonus                         63
deferral_payments            106
deferred_income               96
director_fees                128
exercised_stock_options       43
expenses                      50
from_messages                 58
from_poi_to_this_person       58
from_this_person_to_poi       58
loan_advances                141
long_term_incentive           79
other                         53
poi                            0
restricted_stock              35
restricted_stock_deferred    127
salary                        50
shared_receipt_with_poi       58
to_messages                   58
total_payments                21
total_stock_value             19
dtype: int64

This data set has a lot of missing values, especially for many of the financial features.  Since there are so few records, it would not be appropriate to remove too many rows or columns from this data set.  However, 'loan_advances', 'director_fees', and 'restricted_stock_deferred' have too many missing values to be considered valueable to a model.

In [16]:
df.isna().sum(axis = 1).sort_values(ascending = False)[:10]

LOCKHART EUGENE E         19
WROBEL BRUCE              17
WODRASKA JOHN             17
GRAMM WENDY L             17
WHALEY DAVID A            17
SCRIMSHAW MATTHEW         17
SAVAGE FRANK              16
GILLIS JOHN               16
WAKEHAM JOHN              16
CHRISTODOULOU DIOMEDES    16
dtype: int64

'LOCKHART EUGENE E' has all fields missing except for poi, so he has been removed from the data set.

In [17]:
df.drop('LOCKHART EUGENE E', inplace = True)

In [18]:
# Fill in missing values with 0
df = df.fillna(0)

Many of the financial features have a lot of missing values.  For instance, less than 15% of records have non-null values for the features ‘director_fees’, ‘loan_advances’, and ‘restricted_stock_deferred’.  Many of the nulls in the financial features are due to errors in data wrangling.  In the data source, many zero values are represented as ‘-‘, and were incorrectly interpreted as ‘NaN’.  Additionally, many of the features are aggregations of others, so using imputation to replace missing values would affect multiple features.  Hence, I chose to replace all missing values with zeros, except email addresses which were replaced with an empty string.

In [19]:
# Number of pois in the data set
df['poi'].sum()

18

There are only 18 positive instances out of 146 total records in the data set which means that this model will have to deal with class imbalance. 

## Feature Engineering

In [20]:
# These feature describes the proportion of communication that involves persons of interest
df['prop_poi_communication'] = (df['from_poi_to_this_person'] + df['from_this_person_to_poi']) / \
                               (df['to_messages'] + df['from_messages'])
df['shared_receipt_with_poi_prop'] = df['shared_receipt_with_poi'] / (df['to_messages'] + df['from_messages'])

# These features describe the proportion of total_payments that each individual feature is
df['bonus_prop'] = df['bonus'] / df['total_payments']
df['other_prop'] = df['other'] / df['total_payments']
df['expenses_prop'] = df['expenses'] / df['total_payments']

# These features describe the proportion of total_stock_value that each individual feature is
df['exercised_stock_options_prop'] = df['exercised_stock_options'] / df['total_stock_value']
df['restricted_stock_prop'] = df['restricted_stock'] / df['total_stock_value']

# Get new features list
all_features = [feat for feat in list(df.columns) if feat != 'poi']
all_features

['bonus',
 'deferral_payments',
 'deferred_income',
 'director_fees',
 'exercised_stock_options',
 'expenses',
 'from_messages',
 'from_poi_to_this_person',
 'from_this_person_to_poi',
 'loan_advances',
 'long_term_incentive',
 'other',
 'restricted_stock',
 'restricted_stock_deferred',
 'salary',
 'shared_receipt_with_poi',
 'to_messages',
 'total_payments',
 'total_stock_value',
 'prop_poi_communication',
 'shared_receipt_with_poi_prop',
 'bonus_prop',
 'other_prop',
 'expenses_prop',
 'exercised_stock_options_prop',
 'restricted_stock_prop']

In [21]:
# After possible division by zero fill inf and -inf with 0
df = df.fillna(0)

## Feature Selection

In [31]:
feature_selector = SelectKBest(f_classif, k='all').fit(df[all_features], df[target])
feat_scores = list(zip(feature_selector.scores_, df[all_features].columns[feature_selector.get_support()]))
feat_scores.sort(key=lambda tup: tup[0], reverse = True)
feat_scores

[(24.815079733218194, 'exercised_stock_options'),
 (24.182898678566879, 'total_stock_value'),
 (20.792252047181535, 'bonus'),
 (20.715596247559954, 'bonus_prop'),
 (18.289684043404513, 'salary'),
 (11.458476579280369, 'deferred_income'),
 (9.9221860131898225, 'long_term_incentive'),
 (9.736968287824979, 'shared_receipt_with_poi_prop'),
 (9.2128106219771002, 'restricted_stock'),
 (8.7727777300916756, 'total_payments'),
 (8.589420731682381, 'shared_receipt_with_poi'),
 (7.1840556582887247, 'loan_advances'),
 (6.0941733106389453, 'expenses'),
 (5.399370288094401, 'prop_poi_communication'),
 (5.2434497133749582, 'from_poi_to_this_person'),
 (4.1874775069953749, 'other'),
 (2.3826121082276739, 'from_this_person_to_poi'),
 (2.1263278020077054, 'director_fees'),
 (1.6463411294420076, 'to_messages'),
 (1.0901571696328429, 'restricted_stock_prop'),
 (1.0684733217820632, 'other_prop'),
 (0.22461127473600989, 'deferral_payments'),
 (0.16970094762175533, 'from_messages'),
 (0.065499652909942141, '

In [32]:
# List of the features in order of their importance
feats_sorted = [tup[1] for tup in feat_scores]